In [111]:
# Import libraries
import datetime as dt
import json
import requests as rq
#import bs4 as bs
#from selenium import webdriver
import pandas as pd
import numpy as np
import pybit as pbt
from pybit import usdc_options
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

Достаю для начала все доступные на данный момент даты исполнения и страйки:

In [112]:
key_symbols='https://api-testnet.bybit.com/option/usdc/openapi/public/v1/symbols'
data=rq.get(key_symbols).json()
dataList=[] #достаю словари с необходимым полем symbol
for i in data['result']['dataList']:
                dataList.append(i)           

In [113]:
symbols=[]
for j in dataList:
    symbols.append(j['symbol'])

Достаю названия доступных на данный момент опционов:

Новый код (через pybit)

In [114]:
session = usdc_options.HTTP(
    endpoint='https://api.bybit.com', 
    api_key='...',
    api_secret='...'
)

In [115]:
t1=dt.datetime.now().timestamp()
df=[]
for k in symbols:
    df.append(session.latest_information_for_symbol(symbol=k)['result'])
t2 = dt.datetime.now().timestamp()

print('Processing time:', t2 - t1) #100-200 секунд (зависит от интернета)

Processing time: 134.17207407951355


In [116]:
df1=pd.json_normalize(df) #привожу к формату dataframe
#Удаляю строки с пустыми значениями:
df1.replace('',np.nan,inplace=True)
df1.dropna(subset='symbol', inplace=True)

In [117]:
#оставляю только нужные столбцы:
df2=df1[['symbol','bid','ask','markPrice','markPriceIv']]
df2[['asset','contract_date','strike','option_type']]=df2['symbol'].str.split('-',expand=True)
df2.drop('symbol', axis=1,inplace=True)
df2.set_index('strike',inplace=True)
df2

,bid,ask,markPrice,markPriceIv,asset,contract_date,option_type
strike,,,,,,,
300000,0,20,0.16188168,1.1429,BTC,30DEC22,C
300000,5,0,277706.74179359,1.1429,BTC,30DEC22,P
250000,0,15,0.3249008,1.1109,BTC,30DEC22,C
250000,227600,240030,227706.9049008,1.1109,BTC,30DEC22,P
200000,5,15,0.75439071,1.0697,BTC,30DEC22,C
...,...,...,...,...,...,...,...
19000,0,20,0.27075847,1.2549,BTC,13SEP22,P
18000,0,0,4233.93803449,1.488,BTC,13SEP22,C
18000,0,20,0.06413449,1.488,BTC,13SEP22,P


In [120]:
#Проверка
#df2.loc[(df2['contract_date']=='30DEC22')&(df2.index=='300000')]